In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback
import matplotlib.pyplot as plt

c:\Users\aliki\anaconda3\envs\NLPProject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_excel(r'C:\Users\aliki\Desktop\NLP\vol_3\test_data_2.xlsx')

In [4]:
df.head

<bound method NDFrame.head of                                               comments  label
0                                   Bale's Last Game 💔      0
1                           Rashford who needs ronaldo      0
2                              Smooth goal by Rashford      0
3               England siptted on Wales than on USA..      0
4                                I felt bad for Bale 😢      0
..                                                 ...    ...
114  Women have no place in football. Stick to the ...      1
115  You ape from niggaland are a disgrace to the s...      1
116  I hope someone injures you in the next match. ...      1
117  Football isn't for monkey like you. Go back to...      1
118  You nigga are ruining the game for everyone el...      1

[119 rows x 2 columns]>

In [7]:
X = list(data["comments"])
y = list(data["label"])

In [8]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [9]:
X_tokenized = tokenizer(X, padding=True, truncation=True, max_length=512)

In [5]:
model_path = "output_1/checkpoint-500"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

In [10]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

test_dataset = Dataset(X_tokenized, y)

In [11]:
def compute_metrics(p, l):
    pred = p
    labels = l
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [12]:
test_trainer = Trainer(model)

In [13]:
raw_pred_1, _, _ = test_trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 119
  Batch size = 8
100%|██████████| 15/15 [00:06<00:00,  2.32it/s]


In [14]:
compute_metrics(raw_pred_1, y)

{'accuracy': 0.5378151260504201,
 'precision': 0.1702127659574468,
 'recall': 0.3333333333333333,
 'f1': 0.22535211267605634}

In [15]:
y_pred_1 = np.argmax(raw_pred_1, axis=1)
y_pred_1 = list(y_pred_1)
confusion_matrix(y, y_pred_1)

array([[56, 39],
       [16,  8]], dtype=int64)

In [16]:
df_1 = pd.DataFrame

In [17]:
df_1 = df

In [18]:
df_1["prediction_1"] = y_pred_1

In [19]:
df_1.head

<bound method NDFrame.head of                                               comments  label  prediction_1
0                                   Bale's Last Game 💔      0             1
1                           Rashford who needs ronaldo      0             0
2                              Smooth goal by Rashford      0             0
3               England siptted on Wales than on USA..      0             1
4                                I felt bad for Bale 😢      0             1
..                                                 ...    ...           ...
114  Women have no place in football. Stick to the ...      1             0
115  You ape from niggaland are a disgrace to the s...      1             1
116  I hope someone injures you in the next match. ...      1             0
117  Football isn't for monkey like you. Go back to...      1             0
118  You nigga are ruining the game for everyone el...      1             0

[119 rows x 3 columns]>

In [20]:
df[df["label"] != df["prediction_1"]]

,comments,label,prediction_1
0,Bale's Last Game 💔,0,1
3,England siptted on Wales than on USA..,0,1
4,I felt bad for Bale 😢,0,1
11,bad luck for England they deserved the win aga...,0,1
14,rashford is the best lw in the world,0,1
16,Rashnaldo,0,1
21,Rashford 🚀🔥,0,1
22,harry maguire almost makng that own goal,0,1
23,Kane might have missed the penalty but he was ...,0,1
25,The hole team of England is trash and I hate kane,1,0


In [21]:
model_path = "output_2/checkpoint-500"
model_2 = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

loading configuration file output_2/checkpoint-500\config.json
Model config BertConfig {
  "_name_or_path": "output_1/checkpoint-500",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file output_2/checkpoint-500\pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForS

In [22]:
test_trainer_2 = Trainer(model_2)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
raw_pred_2, _, _ = test_trainer_2.predict(test_dataset)

***** Running Prediction *****
  Num examples = 119
  Batch size = 8
100%|██████████| 15/15 [00:06<00:00,  2.26it/s]


In [24]:
compute_metrics(raw_pred_2, y)

{'accuracy': 0.8487394957983193,
 'precision': 0.6666666666666666,
 'recall': 0.5,
 'f1': 0.5714285714285715}

In [25]:
y_pred_2 = np.argmax(raw_pred_2, axis=1)
y_pred_2 = list(y_pred_2)
confusion_matrix(y, y_pred_2)

array([[89,  6],
       [12, 12]], dtype=int64)

In [26]:
df["prediction_2"] = y_pred_2

In [27]:
df[df["label"] != df["prediction_2"]]

,comments,label,prediction_1,prediction_2
6,England vs mini UK😂🎉,1,1,0
25,The hole team of England is trash and I hate kane,1,0,0
43,Jude f#%^*ng Bellingham…. What a player!!!❤,0,0,1
46,Rashford the GOAT!!!,0,0,1
48,rashford is the goat but its messi,0,1,1
64,Where's the pass to Rashford goal #3 what an i...,0,0,1
68,Wales players showed no heart no desire. They ...,1,1,0
74,where is haters now?!,0,0,1
88,Parry Pane always doing his best to stay troph...,1,0,0
95,My goat maguire,0,0,1


In [28]:
model_path = "output_3/checkpoint-500"
model_3 = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

loading configuration file output_3/checkpoint-500\config.json
Model config BertConfig {
  "_name_or_path": "output_2/checkpoint-500",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file output_3/checkpoint-500\pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForS

In [29]:
test_trainer_3 = Trainer(model_3)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [31]:
raw_pred_3, _, _ = test_trainer_3.predict(test_dataset)

***** Running Prediction *****
  Num examples = 119
  Batch size = 8
100%|██████████| 15/15 [00:06<00:00,  2.40it/s]


In [32]:
compute_metrics(raw_pred_3, y)

{'accuracy': 0.8571428571428571,
 'precision': 0.8181818181818182,
 'recall': 0.375,
 'f1': 0.5142857142857142}

In [33]:
y_pred_3 = np.argmax(raw_pred_3, axis=1)
y_pred_3 = list(y_pred_3)
confusion_matrix(y, y_pred_3)

array([[93,  2],
       [15,  9]], dtype=int64)

In [34]:
df["prediction_3"] = y_pred_3

In [35]:
df[df["label"] != df["prediction_3"]]

,comments,label,prediction_1,prediction_2,prediction_3
3,England siptted on Wales than on USA..,0,1,0,1
6,England vs mini UK😂🎉,1,1,0,0
25,The hole team of England is trash and I hate kane,1,0,0,0
43,Jude f#%^*ng Bellingham…. What a player!!!❤,0,0,1,1
68,Wales players showed no heart no desire. They ...,1,1,0,0
88,Parry Pane always doing his best to stay troph...,1,0,0,0
97,Africa vs africa,1,1,0,0
101,I hate Ronaldo,1,0,1,0
105,why dont you just go back to where you came from,1,0,0,0
106,the more i watch the more i want to kill,1,1,0,0


In [37]:
df.tail(18)

,comments,label,prediction_1,prediction_2,prediction_3
101,I hate Ronaldo,1,0,1,0
102,these scums deserved that,1,1,1,1
103,i cant understand these pricks,1,0,1,1
104,this nigga has to leave,1,0,1,1
105,why dont you just go back to where you came from,1,0,0,0
106,the more i watch the more i want to kill,1,1,0,0
107,mann this was a shitty team,1,1,1,1
108,did you see those untalented bitches,1,1,1,1
109,i gotta say that I hope your career crashes an...,1,0,0,0
110,Someone needs to break your legs on the field....,1,0,0,0


In [56]:
total = y_pred_1
for i in range (119):
    k = y_pred_1[i] + y_pred_2[i] + y_pred_3[i]
    total[i] = 0
    if k > 1:
        total[i] = 1
        

In [57]:
df['total'] = total

In [58]:
df.tail(18)

,comments,label,prediction_1,prediction_2,prediction_3,total
101,I hate Ronaldo,1,0,1,0,0
102,these scums deserved that,1,1,1,1,1
103,i cant understand these pricks,1,0,1,1,1
104,this nigga has to leave,1,0,1,1,1
105,why dont you just go back to where you came from,1,0,0,0,0
106,the more i watch the more i want to kill,1,1,0,0,0
107,mann this was a shitty team,1,1,1,1,1
108,did you see those untalented bitches,1,1,1,1,1
109,i gotta say that I hope your career crashes an...,1,0,0,0,0
110,Someone needs to break your legs on the field....,1,0,0,0,0


In [62]:
confusion_matrix(y, total)

array([[92,  3],
       [15,  9]], dtype=int64)

In [61]:
df[df["label"] != df["total"]]

,comments,label,prediction_1,prediction_2,prediction_3,total
3,England siptted on Wales than on USA..,0,1,0,1,1
6,England vs mini UK😂🎉,1,1,0,0,0
25,The hole team of England is trash and I hate kane,1,0,0,0,0
43,Jude f#%^*ng Bellingham…. What a player!!!❤,0,0,1,1,1
48,rashford is the goat but its messi,0,1,1,0,1
68,Wales players showed no heart no desire. They ...,1,1,0,0,0
88,Parry Pane always doing his best to stay troph...,1,0,0,0,0
97,Africa vs africa,1,1,0,0,0
101,I hate Ronaldo,1,0,1,0,0
105,why dont you just go back to where you came from,1,0,0,0,0
